<a href="https://colab.research.google.com/github/arniery/thesis/blob/main/inference2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text-to-Speech Accent Conversion with Tacotron2, AutoVC, and HifiGAN

This is an English female voice TTS demo using open source projects [BogiHsu/Tacotron2-PyTorch](https://github.com/BogiHsu/Tacotron2-PyTorch) and [jik876/hifi-gan](https://github.com/jik876/hifi-gan).

Please enable GPU acceleration in Colab before you start running the code.

## Set up environment

In [ ]:
!git clone https://github.com/BogiHsu/Tacotron2-PyTorch.git
!pip install -r Tacotron2-PyTorch/requirements.txt
!git clone https://github.com/jik876/hifi-gan.git
!mkdir -p mel_files

Cloning into 'Tacotron2-PyTorch'...
remote: Enumerating objects: 395, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 395 (delta 96), reused 98 (delta 80), pack-reused 263 (from 1)
Receiving objects: 100% (395/395), 2.97 MiB | 7.13 MiB/s, done.
Resolving deltas: 100% (220/220), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 10.0 MB/s eta 0:00:00
Cloning into 'hifi-gan'...
remote: Enumerating objects: 48, done.
remote: Total 48 (delta 0), reused 0 (delta 0), pack-reused 48 (from 1)
Receiving objects: 100% (48/48), 620.94 KiB | 2.14 MiB/s, done.
Resolving deltas: 100% (20/20), done.


## Download Tacotron2 pretrained model

In [ ]:
!wget https://github.com/BogiHsu/Tacotron2-PyTorch/releases/download/lj-200k-b512/ckpt_200000 -O Tacotron2-PyTorch/ckpt_200000

--2025-08-19 18:42:12--  https://github.com/BogiHsu/Tacotron2-PyTorch/releases/download/lj-200k-b512/ckpt_200000
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/189844891/a8fe7134-b9f9-4152-b571-386f437a7c87?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-08-19T19%3A21%3A46Z&rscd=attachment%3B+filename%3Dckpt_200000&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-08-19T18%3A21%3A45Z&ske=2025-08-19T19%3A21%3A46Z&sks=b&skv=2018-11-09&sig=vPXTHBtwsDY5STd9A86YY0ymcwX53Y8HNk42LrLmhFg%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1NTYyOTIzMiwibmJmIjoxNzU1NjI4OTMyLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi

## Download HifiGAN pretrained model

In [ ]:
import gdown
dl = {'hifi-gan/config.json': 'https://drive.google.com/u/1/uc?id=1aDh576AEYA5eTjhx7sew1qcCM_Y526jc&export=download',
      'hifi-gan/generator_v1': 'https://drive.google.com/u/1/uc?id=14NENd4equCBLyyCSke114Mv6YR_j_uFs&export=download'}
for k in dl:
    gdown.download(dl[k], k)

Downloading...
From: https://drive.google.com/u/1/uc?id=1aDh576AEYA5eTjhx7sew1qcCM_Y526jc&export=download
To: /content/hifi-gan/config.json
100%|██████████| 795/795 [00:00<00:00, 3.00MB/s]
Downloading...
From: https://drive.google.com/u/1/uc?id=14NENd4equCBLyyCSke114Mv6YR_j_uFs&export=download
To: /content/hifi-gan/generator_v1
100%|██████████| 55.8M/55.8M [00:01<00:00, 50.4MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/autovc_data/autovc.ckpt ./
!cp -r /content/drive/MyDrive/autovc_data/spmel ./
!cp -r /content/drive/MyDrive/autovc_data/wav16k ./


### modifying/fine-tuning hifi-gan

In [ ]:
import os
import random

# Set seed for reproducibility
random.seed(42)

# Paths to your AutoVC-generated data
mel_root = '/content/spmel/female_speaker'
wav_root = '/content/wav16k/female_speaker'
filelist_dir = '/content/hifi-gan/filelists'
train_file = os.path.join(filelist_dir, 'training.txt')
val_file = os.path.join(filelist_dir, 'validation.txt')

lines = []

# Pair up mel and wav files
for fname in sorted(os.listdir(mel_root)):
    if not fname.endswith('.npy'):
        continue
    mel_path = os.path.join(mel_root, fname)

    # FIX: safer wav file naming
    wav_fname = os.path.splitext(fname)[0] + '.wav'
    wav_path = os.path.join(wav_root, wav_fname)

    if os.path.exists(wav_path):
        lines.append(f"{mel_path}|{wav_path}")
    else:
        print(f"Warning: Missing wav file for {fname}")

# Shuffle and split (95% train, 5% validation)
random.shuffle(lines)
split_idx = int(0.95 * len(lines))
train_lines = lines[:split_idx]
val_lines = lines[split_idx:]

# Write to file
os.makedirs(filelist_dir, exist_ok=True)

with open(train_file, 'w') as f:
    f.write('\n'.join(train_lines))

with open(val_file, 'w') as f:
    f.write('\n'.join(val_lines))

print(f"Training file saved to: {train_file}, total: {len(train_lines)}")
print(f"Validation file saved to: {val_file}, total: {len(val_lines)}")


Training file saved to: /content/hifi-gan/filelists/training.txt, total: 6213
Validation file saved to: /content/hifi-gan/filelists/validation.txt, total: 328


In [ ]:
import json

flat_config = {
  "batch_size": 32,                # Double your current batch size if GPU memory allows
  "learning_rate": 0.0002,
  "adam_b1": 0.8,
  "adam_b2": 0.99,
  "lr_decay": 0.999,
  "seed": 1234,
  "fp16_run": True,                # Enable mixed-precision for faster training
  "sampling_rate": 16000,
  "segment_size": 4096,            # Smaller segments = faster per-step computation
  "filter_length": 1024,
  "hop_size": 256,
  "win_size": 1024,
  "n_fft": 1024,
  "num_mels": 80,
  "fmin": 90,
  "fmax": 7600,
  "fmax_for_loss": None,
  "upsample_rates": [8, 8, 2, 2],
  "upsample_kernel_sizes": [16, 16, 4, 4],
  "upsample_initial_channel": 512,
  "resblock": "1",
  "resblock_kernel_sizes": [3, 7, 11],
  "resblock_dilation_sizes": [[1, 3, 5], [1, 3, 5], [1, 3, 5]],
  "resblock_initial_channel": 256,
  "num_workers": 4,                # DataLoader workers
  "num_gpus": 1,
  "dist_config": {
    "dist_backend": "nccl",
    "dist_url": "tcp://localhost:54321",
    "world_size": 1
  }
}

with open("/content/hifi-gan/config_autovc.json", "w") as f:
    json.dump(flat_config, f, indent=4)

print("Flat config saved to /content/hifi-gan/config_autovc.json")

Flat config saved to /content/hifi-gan/config_autovc.json


In [ ]:
!cp -r /content/drive/MyDrive/autovc_data /content/

In [ ]:
# fine-tuning
%cd /content/hifi-gan

!python train.py \
  --config config_autovc.json \
  --input_training_file filelists/training.txt \
  --input_validation_file filelists/validation.txt \
  --input_wavs_dir /content/wav16k/female_speaker \
  --input_mels_dir /content/spmel/female_speaker \
  --training_epochs 12


/content/hifi-gan
2025-08-17 21:24:59.454058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755465899.752803   15261 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755465899.834352   15261 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755465900.444271   15261 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755465900.444323   15261 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755465900.444327   15261 computation_placer.cc:177] comp

In [ ]:
!head -n 5 /content/hifi-gan/filelists/training.txt

/content/drive/MyDrive/autovc_data/spmel/female_speaker/train_hindifullfemale_02131_16k.npy|/content/drive/MyDrive/autovc_data/wav16k/female_speaker/train_hindifullfemale_02131_16k.wav
/content/drive/MyDrive/autovc_data/spmel/female_speaker/train_hindifullfemale_01097_16k.npy|/content/drive/MyDrive/autovc_data/wav16k/female_speaker/train_hindifullfemale_01097_16k.wav
/content/drive/MyDrive/autovc_data/spmel/female_speaker/train_hindifullfemale_03475_16k.npy|/content/drive/MyDrive/autovc_data/wav16k/female_speaker/train_hindifullfemale_03475_16k.wav
/content/drive/MyDrive/autovc_data/spmel/female_speaker/train_hindifullfemale_05334_16k.npy|/content/drive/MyDrive/autovc_data/wav16k/female_speaker/train_hindifullfemale_05334_16k.wav
/content/drive/MyDrive/autovc_data/spmel/female_speaker/train_hindifullfemale_01716_16k.npy|/content/drive/MyDrive/autovc_data/wav16k/female_speaker/train_hindifullfemale_01716_16k.wav


In [ ]:
ls /content

autovc.ckpt  hifi-gan/   sample_data/  Tacotron2-PyTorch/
drive/       mel_files/  spmel/        wav16k/


## AutoVC Implementation



In [ ]:
!git clone https://github.com/auspicious3000/autovc.git
%cd autovc
# !pip install -r requirements.txt  # If there's a requirements file

# Add print statement to model_vc.py to debug freq
!sed -i "/def forward(self, x, c_org):/a\ \ \ \ \ \ \ \ print(f'Encoder self.freq: {self.freq}')" model_vc.py
# Add print statement in Generator's forward to check c_org shape
!sed -i "/codes = self.encoder(x, c_org)/i\ \ \ \ \ \ \ \ print(f'Generator forward - shape of c_org before encoder: {c_org.shape}')" model_vc.py

Cloning into 'autovc'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 116 (delta 4), reused 9 (delta 4), pack-reused 104 (from 1)
Receiving objects: 100% (116/116), 8.18 MiB | 12.39 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/autovc


## 1. Load Tacotron2 (Text → Mel)

In [ ]:
import sys
sys.path.append("/content/Tacotron2-PyTorch")

In [ ]:
%cd Tacotron2-PyTorch
!python3 inference.py --ckpt_pth=ckpt_200000 --text='Tacotron is awesome.' --npy_pth=../mel_files/1
%cd ../

/content/Tacotron2-PyTorch
Terminated by gate.
/content


## 2. Run AutoVC (Source Mel → Target Mel)

In [ ]:
import sys
sys.path.append('/content/autovc')  # adjust if you cloned elsewhere

from model_vc import Generator
G = Generator(dim_neck=32, dim_emb=256, dim_pre=512, freq=32).cuda()

# Load the model weights
ckpt = torch.load('/content/autovc.ckpt', map_location='cpu')
G.load_state_dict(ckpt['model'])
G.eval()

import torch
import numpy as np
import os

def run_autovc(generator, dim_emb, source_mel_path, output_path, ref_mel_path=None, emb_org=None, emb_trg=None):
    def pad_seq(x, base=32):
        len_out = int(base * np.ceil(float(x.shape[1]) / base))
        len_pad = len_out - x.shape[1]
        if len_pad == 0:
            return x
        pad = np.zeros((x.shape[0], len_pad))
        return np.concatenate((x, pad), axis=1)

    content = np.load(source_mel_path)
    print(f"Shape of mel before padding: {content.shape}")
    content = pad_seq(content.T).T
    print(f"Shape of mel after padding: {content.shape}")
    content_tensor = torch.from_numpy(content).unsqueeze(0).cuda().float()

    # Generate embedding
    if ref_mel_path:
        ref = np.load(ref_mel_path)
        ref_tensor = torch.FloatTensor(ref).unsqueeze(0).cuda().float()
        with torch.no_grad():
            _, ref_emb = generator(ref_tensor, ref_tensor, ref_tensor)
    elif emb_trg is not None:
        ref_emb = emb_trg.unsqueeze(0).cuda().float()
    else:
        ref_emb = torch.randn(1, dim_emb).cuda().float()

    with torch.no_grad():
        c_org = ref_emb
        c_trg = ref_emb
        converted_mel, _, _ = generator(content_tensor, c_org, c_trg)

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    np.save(output_path, converted_mel.squeeze(0).cpu().numpy())
    print(f"Saved converted mel to: {output_path}")

source_mel = '/content/drive/MyDrive/autovc_data/spmel/female_speaker/train_hindifullfemale_00001_16k.npy'
output_mel = '/content/converted_mels/train_hindifullfemale_00001_16k_converted.npy'

run_autovc(
    generator=G,
    dim_emb=256,
    source_mel_path=source_mel,
    output_path=output_mel
)

Shape of mel before padding: (272, 80)
Shape of mel after padding: (288, 80)


RuntimeError: Given groups=1, weight of size [512, 336, 5], expected input[1, 544, 80] to have 336 channels, but got 544 channels instead

## 3. HiFi-GAN Vocoder (Mel → Wav)

In [ ]:
# @title
%cd hifi-gan
!python3 inference_e2e.py --checkpoint_file g_00002000 --input_mels_dir ../spmel/female_speaker --output_dir ../wav_files
%cd ../

/content/hifi-gan
Initializing Inference Process..
/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Loading 'g_00002000'
Complete.
Removing weight norm...
Traceback (most recent call last):
  File "/content/hifi-gan/inference_e2e.py", line 89, in <module>
    main()
  File "/content/hifi-gan/inference_e2e.py", line 85, in main
    inference(a)
  File "/content/hifi-gan/inference_e2e.py", line 50, in inference
    y_g_hat = generator(x)
              ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py", line 1773, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py", line 1784, in _call_impl
    return forward_call(*args, **kwargs)
           

## Play synthesized wav file
### 4. Full pipeline

In [ ]:
!echo "" > /content/Tacotron2-PyTorch/utils/__init__.py

In [ ]:
import os, torch, numpy as np, librosa
from scipy.io.wavfile import write as wav_write

# --- CONFIG ---
TEXT = "Tacotron is awesome."
REF_WAV = "/content/wav16k/female_speaker/train_hindifullfemale_03157_16k.wav"   # <-- put your reference wav here
OUT_WAV = "/content/final_output.wav"

# === Step 1: Tacotron2 ===
print("Step 1: Tacotron2...")
taco_mel = tts_to_mel(TEXT)  # (80, T)
print("Tacotron2 mel shape:", taco_mel.shape)

# === Step 2: AutoVC ===
# Step 2: Voice conversion
autovc_mel = run_autovc(taco_mel)

# === Step 3: HiFi-GAN ===
print("Step 3: HiFi-GAN...")
wav = mel_to_wav(autovc_mel)

# Save WAV
wav_write(OUT_WAV, 22050, wav)
print("Saved final output to:", OUT_WAV)


Step 1: Tacotron2...
Terminated by gate.
Tacotron2 mel shape: (80, 135)


TypeError: run_autovc() missing 3 required positional arguments: 'dim_emb', 'source_mel_path', and 'output_path'